In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Sun Jan  7 13:06:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              47W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Data Augmentation

In [ ]:
"""
Data Augmentation
"""

from imgaug import augmenters as iaa
import numpy as np
import cv2
import json
import glob as gb

#--- a function for image augmentation ---
def data_aug(img, flags1, flags2):

  img_shape = img.shape
  images = np.random.randint(0, 1, (1, img_shape[0], img_shape[1], img_shape[2]), dtype=np.uint8)
  images[0] = img
  seq1 = iaa.Sequential([
      iaa.Sometimes(flags1[0], iaa.GaussianBlur(sigma=(2, 11))),

      iaa.Sometimes(flags1[1], iaa.MedianBlur(k=(3, 25))),

      iaa.Sometimes(flags1[2], iaa.Add((-10, 10))),

      iaa.Sometimes(flags1[3], iaa.WithColorspace(to_colorspace="HSV",
                                                  children=iaa.WithChannels(channels=[1],
                                                                            children=iaa.Add((-5, 5))))),

      iaa.Sometimes(flags1[4], iaa.WithColorspace(to_colorspace="HSV",
                                                  children=iaa.WithChannels(channels=[2],
                                                                            children=iaa.Add((-5, 5))))),

      iaa.Sometimes(flags1[5], iaa.Multiply((0.9, 1.1))),

      iaa.Sometimes(flags1[6], iaa.Sharpen(alpha=(0, 1), lightness=(0.75, 1.25))),

      iaa.Sometimes(flags1[7], iaa.AdditiveGaussianNoise(loc=0, scale=(0, 5))),

      iaa.Sometimes(flags1[8], iaa.SaltAndPepper(p=(0, 0.005))),

      #iaa.Sometimes(flags1[9], iaa.ContrastNormalization((0.85, 1.15))),

      iaa.Sometimes(flags1[10], iaa.WithColorspace(to_colorspace="HSV",
                                                    children=iaa.WithChannels(channels=[1, 2],
                                                                              children=iaa.Add((-5, 5))))),
      iaa.Sometimes(0, iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.15)),
      iaa.Sometimes(0, iaa.Emboss(alpha=(0, 1), strength=(0, 4))),
      iaa.Sometimes(flags1[13], iaa.GaussianBlur(sigma=(4, 11))),
      iaa.Sometimes(flags2[0], iaa.Fliplr(1)),
      iaa.Sometimes(flags2[1], iaa.Flipud(1)),
      iaa.Sometimes(flags2[2], iaa.Affine(rotate=(-90, 90) , mode= 'edge')),
      iaa.Sometimes(flags2[3], iaa.Affine(scale=(0.8, 1.2) , mode='edge'))

  ])
  images_aug = seq1.augment_images(images)
  img_aug1 = images_aug[0]
  return img_aug1

# ---this function manage number of Augmenations for each class---
def manage_aug(imgs, n_fold):


  flag1 = [0] * 14
  flag2 = [0] * 4
  imgs_aug = []
  for img in imgs:

    # create n-fold random numbers between 1 to 15 without repeat
    imgs_aug.append(img)
    random_Nums = []
    len_rand_Nums = 1
    while len_rand_Nums < n_fold:
      temp = np.random.randint(1 , 16)
      if temp not in random_Nums :
        random_Nums.append(temp)
        len_rand_Nums+=1

    # This Section aim to augment the image
    for j in range(len(random_Nums)) :
      num = random_Nums[j]

      num_bin = bin(num).replace('0b' , '') # converting number_aug from decimal to binary to serve flag2

      # fill num_bin by zero because its length must be 4
      if len(num_bin) == 1 :
        num_bin = '000' + num_bin
      elif len(num_bin) == 2 :
        num_bin = '00' + num_bin
      elif len(num_bin) == 3 :
        num_bin = '0' + num_bin

      flag2[0] , flag2[1] , flag2[2] , flag2[3] = int(num_bin[3]) , int(num_bin[2])  , int(num_bin[1]) , int(num_bin[0])

      img_aug = data_aug(img, flags1 = flag1 , flags2= flag2)
      imgs_aug.append(img_aug)
  return imgs_aug



# Installing vit-keras and tensorflow_addons libraries

In [ ]:
! pip install tensorflow_addons
! pip install vit-keras

# Deep learning models

In [ ]:
""""
Deep Learning Models
"""
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as resnet_prep
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input as mob_prep
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input as dens_prep
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_prep
from vit_keras import vit, utils as vit_utils
from vit_keras.vit import preprocess_inputs as vit_prep
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, SGD



def vgg16(input_shape=(224, 224, 3), base_trainable=False, weights="imagenet", optimizer='adam'):

  ## Loading VGG16 model
  base_model = VGG16(weights=weights, include_top=False, input_shape=input_shape)
  base_model.trainable = True ## Not trainable weights
  # for layer in base_model.layers[:-6]:
  #  layer.trainable = False

  data_augmentation = models.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  ])

  flatten_layer = layers.Flatten()
  dense_layer_1 = layers.Dense(1024, activation='relu')
  # dense_layer_2 = layers.Dense(512, activation='relu')
  prediction_layer = layers.Dense(3, activation='softmax')

  model = models.Sequential([
      layers.RandomFlip("horizontal_and_vertical"),
      layers.RandomRotation(0.2),
      base_model,
      flatten_layer,
      # dense_layer_1,
      #dense_layer_2,
      prediction_layer
  ])


  # model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
  model.compile(optimizer=Adam(learning_rate=0.001), loss=CategoricalCrossentropy() , metrics=['accuracy'])
  # model.compile(optimizer=Adam(lr=0.001), loss=SparseCategoricalCrossentropy(from_logits=True) , metrics=['accuracy'])
  return model


def resnet50(input_shape=(224, 224, 3), base_trainable=False, weights="imagenet", optimizer='adam'):

  ## Loading VGG16 model
  base_model = ResNet50(weights=weights, include_top=False, input_shape=input_shape)
  base_model.trainable = True ## Not trainable weights
  # for layer in base_model.layers[:-6]:
  #  layer.trainable = False

  data_augmentation = models.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  ])

  flatten_layer = layers.Flatten()
  dense_layer_1 = layers.Dense(1024, activation='relu')
  # dense_layer_2 = layers.Dense(512, activation='relu')
  prediction_layer = layers.Dense(3, activation='softmax')

  model = models.Sequential([
      layers.RandomFlip("horizontal_and_vertical"),
      layers.RandomRotation(0.2),
      base_model,
      flatten_layer,
      # dense_layer_1,
      #dense_layer_2,
      prediction_layer
  ])


  # model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
  model.compile(optimizer=Adam(learning_rate=0.001), loss=CategoricalCrossentropy() , metrics=['accuracy'])
  # model.compile(optimizer=Adam(lr=0.001), loss=SparseCategoricalCrossentropy(from_logits=True) , metrics=['accuracy'])
  return model


def mobnetv2(input_shape=(224, 224, 3), base_trainable=False, weights="imagenet", optimizer='adam'):

  ## Loading VGG16 model
  base_model = MobileNetV2(weights=weights, include_top=False, input_shape=input_shape)
  base_model.trainable = True ## Not trainable weights
  # for layer in base_model.layers[:-6]:
  #  layer.trainable = False

  data_augmentation = models.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  ])

  flatten_layer = layers.Flatten()
  dense_layer_1 = layers.Dense(1024, activation='relu')
  # dense_layer_2 = layers.Dense(512, activation='relu')
  prediction_layer = layers.Dense(3, activation='softmax')

  model = models.Sequential([
      layers.RandomFlip("horizontal_and_vertical"),
      layers.RandomRotation(0.2),
      base_model,
      flatten_layer,
      # dense_layer_1,
      #dense_layer_2,
      prediction_layer
  ])


  # model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
  model.compile(optimizer=Adam(learning_rate=0.001), loss=CategoricalCrossentropy() , metrics=['accuracy'])
  # model.compile(optimizer=Adam(lr=0.001), loss=SparseCategoricalCrossentropy(from_logits=True) , metrics=['accuracy'])
  return model

def densenet121(input_shape=(224, 224, 3), base_trainable=False, weights="imagenet", optimizer='adam'):
  base_model = tf.keras.applications.densenet.DenseNet121(weights=weights, include_top=False, input_shape=input_shape)
  base_model.trainable = True ## Not trainable weights
  # for layer in base_model.layers[:-6]:
  #  layer.trainable = False

  data_augmentation = models.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  ])

  flatten_layer = layers.Flatten()
  dense_layer_1 = layers.Dense(1024, activation='relu')
  # dense_layer_2 = layers.Dense(512, activation='relu')
  prediction_layer = layers.Dense(3, activation='softmax')

  model = models.Sequential([
      layers.RandomFlip("horizontal_and_vertical"),
      layers.RandomRotation(0.2),
      base_model,
      flatten_layer,
      # dense_layer_1,
      #dense_layer_2,
      prediction_layer
  ])


  # model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
  model.compile(optimizer=Adam(learning_rate=0.001), loss=CategoricalCrossentropy() , metrics=['accuracy'])
  # model.compile(optimizer=Adam(lr=0.001), loss=SparseCategoricalCrossentropy(from_logits=True) , metrics=['accuracy'])
  return model



def vit_b16(input_shape=(224, 224, 3), base_trainable=False, weights="imagenet", optimizer='adam'):
  img_size = 224
  base_model = vit.vit_b16(
    image_size=(224, 224),
    activation='sigmoid',
    pretrained=True,
    include_top=False,
    pretrained_top=False)

  model = models.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    base_model,
    layers.Dense(units=3, activation='softmax')])
  model.compile(optimizer=Adam(lr=0.001), loss=CategoricalCrossentropy() , metrics=['accuracy'])

  return model




def model_selector(model_name):
  if model_name=='vgg16':
    my_model = vgg16(input_shape=(224, 224, 3), base_trainable=True, weights="imagenet", optimizer='adam')
    pre_process = vgg_prep
  elif model_name=='resnet50':
    my_model = resnet50(input_shape=(224, 224, 3), base_trainable=True, weights="imagenet", optimizer='adam')
    pre_process = resnet_prep
  elif model_name == 'mobnetv2':
    my_model = mobnetv2(input_shape=(224, 224, 3), base_trainable=True, weights="imagenet", optimizer='adam')
    pre_process = mob_prep
  elif model_name=='densenet121':
    my_model = densenet121(input_shape=(224, 224, 3), base_trainable=True, weights="imagenet", optimizer='adam')
    pre_process = dens_prep
  elif model_name=='vit':
    my_model =  vit_b16()
    pre_process = vit_prep

  return my_model, pre_process





# Loading data from memory

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import glob
from tensorflow.keras.preprocessing.image import load_img



data_dir = '/content/drive/My Drive/Fungal-Fungal-interaction/data/'
code_dir = '/content/drive/My Drive/Fungal-Fungal-interaction/'
well_dir = '/content/drive/My Drive/Fungal-Fungal-interaction/data/wells/'
model_dir = '/content/drive/My Drive/Fungal-Fungal-interaction/models/'
data = pd.read_csv(data_dir + '01_well_data.csv')
raw_paths, lbls = data['img_paths'], data['labels']
name_lbls = {}
x, y = [], []
for i in range(len(raw_paths)):
  img_name = raw_paths[i].split('/')[-2] + '/' + raw_paths[i].split('/')[-1]
  lbl = int(lbls[i])
  # img = cv2.imread(well_dir+img_name)
  # img = cv2.resize(img, dsize=(224, 224))
  img = np.array(load_img(well_dir+img_name, target_size=(224, 224)))
  x.append(img), y.append(lbl)
x, y = np.array(x), np.array(y)
np.random.seed(42)
x = np.random.permutation(x)
np.random.seed(42)
y = np.random.permutation(y)

In [ ]:
#import glob
#pathes = glob.glob('/content/drive/My Drive/Fungal-Fungal-interaction/data/wells/*/')
#print(pathes)
import os
plate_names = [well for well in os.listdir(well_dir)]
np.random.seed(42)
plate_names = np.random.permutation(plate_names)

# Training and evaluation of the models through 5-fold cross-validation
In this block, all models are trained and evaluated, then, the trained models are saved in memory. Also, the performance of each model across five folds are saved in the memory as a dictionary (*.joblib).

In [ ]:

model_names = ['vgg16', 'resnet50', 'mobnetv2', 'densenet121', 'vit']

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_prep
from sklearn.metrics import recall_score as recall, precision_score as precision, f1_score

k = 5
slide = len(x)//k
prec, rec, f1, acc = [], [], [], []
performance = {}
for model_name in model_names:
  try:
    del my_model
  except:
    pass
  perf_kfold = {}
  for i in range(0, k):
    print(model_name, "fold", i, '\n', '#'*50)
    # Specifying what images for test and what images for training and validation
    if i == k-1:
      test_idx = np.arange(i*slide, len(x))
      rest_idx = np.arange(0, i*slide)
    else:
      test_idx = np.arange(i*slide, (i+1)*slide)
      rest_idx = [id for id in range(len(x)) if id not in test_idx]
    x_test, y_test = x[test_idx], y[test_idx]
    x_rest, y_rest = x[rest_idx], y[rest_idx]

    # Augmentation
    num_cls0, num_cls1, num_cls2 = y_rest.tolist().count(0), y_rest.tolist().count(1), y_rest.tolist().count(2)
    cls_max = np.max([num_cls0, num_cls1, num_cls2])
    n_fold0, n_fold1, n_fold2 = (np.round(cls_max/num_cls0))*2, (np.round(cls_max/num_cls1))*2, (np.round(cls_max/num_cls2))*2
    #print(num_cls0, num_cls1, num_cls2, cls_max, n_fold0, n_fold1, n_fold2)
    #break
    #print(y_rest.shape, x_rest.shape)
    #print(type(y_rest))
    x_rest_aug, y_rest_aug = [], []
    for cls in [0, 1, 2]:
      num_cls = y_rest.tolist().count(cls)
      #print(num_cls)
      if cls!=2:
        n_fold = int((np.round(cls_max/num_cls))*2)
      else:
        n_fold = 14
      indexes = np.argwhere(y_rest==cls).reshape(-1)
      #print(indexes.shape)
      imgs = x_rest[indexes]
      img_augs = manage_aug(imgs, n_fold)
      img_augs = manage_aug(img_augs, 2)
      x_rest_aug.extend(img_augs)
      lbls = [cls]*len(img_augs)
      y_rest_aug.extend(lbls)

    np.random.seed(42)
    x_rest = np.random.permutation(x_rest_aug)
    np.random.seed(42)
    y_rest = np.random.permutation(y_rest_aug)

    x_train, y_train = x_rest[:int(0.8*len(y_rest))], y_rest[:int(0.8*len(y_rest))]
    x_val, y_val = x_rest[int(0.8*len(y_rest)):], y_rest[int(0.8*len(y_rest)):]

    try:
      del my_model
    except:
      pass
    my_model, preprocess = model_selector(model_name)
    x_train = preprocess(x_train)
    x_val = preprocess(x_val)
    x_test = preprocess(x_test)
    y_train_cat = to_categorical(y_train, num_classes=3)
    y_val_cat = to_categorical(y_val, num_classes=3)
    y_test_cat = to_categorical(y_test, num_classes=3)

    my_model.build((None, 224, 224, 3))
    my_model.summary()
    es = EarlyStopping(monitor='val_loss', mode='min', patience=20,  restore_best_weights=True)
    history = my_model.fit(x_train, y_train_cat, epochs=200, validation_data=(x_val, y_val_cat), batch_size=16, callbacks=[es], verbose=True)

    res = {}
    train_loss, train_acc, val_loss, val_acc = history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']
    res['training_process'] = [train_loss, train_acc, val_loss, val_acc]

    y_pred = np.argmax(my_model.predict(x_test), axis=1)
    res['true_pred_lbls'] = [y_test, y_pred]

    prec = precision(y_test, y_pred, average=None)
    rec = recall(y_test, y_pred, average=None)
    f = f1_score(y_test, y_pred, average=None)
    res['p_r_f'] = [prec, rec, f]

    acc = my_model.evaluate(x_test, y_test_cat)[1]
    print(prec, '\n', rec, '\n', f)
    res['acc'] = acc

    perf_kfold[str(i)] = res

    my_model.save(model_dir+model_name+'_'+'k'+str(i)+'.keras')
  performance[model_name] = perf_kfold

import joblib
joblib.dump(performance, code_dir+'performance-'+model_name+'.joblib')













vit fold 0 
 ##################################################


/usr/local/lib/python3.10/dist-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 14, 14
  warnings.warn(


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_2 (RandomFlip)  (None, 224, 224, 3)       0         
                                                                 
 random_rotation_2 (RandomR  (None, 224, 224, 3)       0         
 otation)                                                        
                                                                 
 vit-b16 (Functional)        (None, 768)               85798656  
                                                                 
 dense_2 (Dense)             (None, 3)                 2307      
                                                                 
Total params: 85800963 (327.30 MB)
Trainable params: 85800963 (327.30 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/200
411/411 [==============================] - 118s 196ms/step - los

/usr/local/lib/python3.10/dist-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 14, 14
  warnings.warn(


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_3 (RandomFlip)  (None, 224, 224, 3)       0         
                                                                 
 random_rotation_3 (RandomR  (None, 224, 224, 3)       0         
 otation)                                                        
                                                                 
 vit-b16 (Functional)        (None, 768)               85798656  
                                                                 
 dense_3 (Dense)             (None, 3)                 2307      
                                                                 
Total params: 85800963 (327.30 MB)
Trainable params: 85800963 (327.30 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/200
413/413 [==============================] - 118s 196ms/step - los

...

/usr/local/lib/python3.10/dist-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 14, 14
  warnings.warn(


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_5 (RandomFlip)  (None, 224, 224, 3)       0         
                                                                 
 random_rotation_5 (RandomR  (None, 224, 224, 3)       0         
 otation)                                                        
                                                                 
 vit-b16 (Functional)        (None, 768)               85798656  
                                                                 
 dense_5 (Dense)             (None, 3)                 2307      
                                                                 
Total params: 85800963 (327.30 MB)
Trainable params: 85800963 (327.30 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/200
405/405 [==============================] - 117s 195ms/step - los

/usr/local/lib/python3.10/dist-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 14, 14
  warnings.warn(


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_6 (RandomFlip)  (None, 224, 224, 3)       0         
                                                                 
 random_rotation_6 (RandomR  (None, 224, 224, 3)       0         
 otation)                                                        
                                                                 
 vit-b16 (Functional)        (None, 768)               85798656  
                                                                 
 dense_6 (Dense)             (None, 3)                 2307      
                                                                 
Total params: 85800963 (327.30 MB)
Trainable params: 85800963 (327.30 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/200
406/406 [==============================] - 117s 199ms/step - los

['/content/drive/My Drive/Fungal-Fungal-interaction/performanceViT.joblib']

In [ ]:
import joblib
import glob

perf_paths = joblib.joblib(code_dir+'*.joblib')
a = joblib.load(perf_path[0]).update(joblib.load(perf_path[2]))